In [1]:
from bs4 import BeautifulSoup
import requests
import requests.cookies
import http.cookiejar as cookiejar
from importlib import reload
import re
from pprint import pprint
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as page_ec
from selenium.common.exceptions import TimeoutException

In [2]:
from extractors import AddressExtractor, OrgExtractor
from natasha.extractors import Extractor
from natasha.grammars.address import ADDRESS

In [3]:
from yargy import (
    rule,
    not_,
    and_,
    or_,
)
from yargy.interpretation import attribute, fact
from yargy.predicates import (
    eq,
    in_,
    true,
    gram,
    type,
    caseless,
    normalized,
    is_capitalized,
    is_lower,
    is_single,
)
from yargy.relations import (
    gnc_relation,
    case_relation,
    main,
)

from yargy import Parser
from yargy.pipelines import morph_pipeline, caseless_pipeline, pipeline
from yargy.tokenizer import QUOTES, LEFT_QUOTES, RIGHT_QUOTES, GENERAL_QUOTES,Tokenizer

from yargy.tokenizer import MorphTokenizer
TOKENIZER = MorphTokenizer().remove_types('EOL')

from extractors.NamesExtractor import SIMPLE_NAME
from extractors.PersonExtractor import POSITION_NAME
from extractors.SettlementExtractor import RESPUBLIKA, KRAI, OBLAST, AUTO_OKRUG, RAION, GOROD, SELO, POSELOK, DEREVNYA

from yargy.rule.transformators import RuleTransformator

In [4]:
header = {
            'User-Agent': "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; InfoPath.2; .NET CLR 1.1.4322; .NET4.0C; Tablet PC 2.0)",
        }

In [5]:
config = webdriver.DesiredCapabilities.FIREFOX.copy()
config['platform'] = 'LINUX'
options = webdriver.firefox.options.Options()
options.headless = True
driver = webdriver.Remote("http://localhost:4444/wd/hub", desired_capabilities=config, options=options)

In [6]:
def preproc_cookie(cookie_dict):
    avail_names = ['version', 'name', 'value', 'port', 'domain', 'path', 'secure', 'expires', 'discard',
        'comment', 'comment_url', 'rfc2109']
    
    res_cookies = {key : value for key, value in cookie_dict.items() if key in avail_names}
    res_cookies['rest'] = {'HttpOnly': cookie_dict.get('httpOnly', None)}
    return res_cookies
    

In [7]:
cookie_pattern = re.compile(r'document\.cookie\s*=')
def get_page(url):
    with requests.Session() as session:
        session.headers = header
        try:
            resp = session.get(url, timeout=(3, 2.5)) # делаем запрос
#                 print(resp.headers)
        except requests.RequestException as e: # если ошибки при запросе
#             print("Exception {} on {}\n".format(e, url))
            return None
        else:
            soup = BeautifulSoup(resp.content, "html.parser")
            cookies = {}
            for script in soup(["script"]):
                script_text = script.getText(separator='\n', strip=True)
                found = cookie_pattern.search(script_text)
                if found is not None:
                    driver.get(url)
                    for cookie in driver.get_cookies():
                        session.cookies.set(**preproc_cookie(cookie))
                    driver.delete_all_cookies()
                
#                 found = cookie_pattern.findall(script_text)
#                 for match in found:
#                     for cookie in match.split(';'):
#                         cookie = cookie.strip()
#                         if len(cookie) > 0:
#                             var, value = cookie.split('=', 1)
#                             if value is not None:
#                                 cookies[var] = value
            if len(session.cookies) > 0:
                try:
                    resp = session.get(url, timeout=(3, 2.5)) # делаем запрос
        #                 print(resp.headers)
                except requests.RequestException as e: # если ошибки при запросе
        #             print("Exception {} on {}\n".format(e, url))
                    return None
            return resp

In [8]:
res = get_page('http://0058867.com/')
# res = get_page('http://www.example.com/')

In [10]:
res.content

b'<!DOCTYPE html>\r\n<html lang="en">\r\n    <head>\r\n       <meta charset="utf-8">\r\n\t   <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" /> \r\n       <title>\xe6\xbe\xb3\xe9\x97\xa8\xe6\x96\xb0\xe8\x91\xa1\xe4\xba\xac\xe5\xae\x98\xe6\x96\xb9\xe7\xbd\x91-\xe6\xac\xa2\xe8\xbf\x8e\xe6\x82\xa8\xe7\x9a\x84\xe5\x85\x89\xe4\xb8\xb4\xef\xbc\x81www.8867.com</title>\r\n       <meta name="description" content="" />\r\n        <meta name="viewport" content="width=device-width, initial-scale=1.0">\r\n        <!-- Le styles -->\r\n      <link href="/style/share/share.css" media="screen" rel="stylesheet" type="text/css">\n<link href="/style/aomenxinpujingk/css.css?v=1567167378" media="screen" rel="stylesheet" type="text/css">\n<link href="/style/aomenxinpujingk/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon">\r\n        <!-- Scripts -->\r\n        <script type="text/javascript" src="/js/jquery.min.js"></script>\n<script type="text/javascript" src="/style/share/jque

In [5]:
class StripInterpretationTransformator(RuleTransformator):
    def visit_InterpretationRule(self, item):
        return self.visit(item.rule)

NAME = SIMPLE_NAME.transform(StripInterpretationTransformator)
PERSON = POSITION_NAME.transform(StripInterpretationTransformator)

In [6]:
ABBR_TYPE = pipeline([
    'АО',
    'ОАО',
    'ООО',
    'ЗАО',
    'ПАО',
    'ИП',
    'БФ',
    'ГСК',
    'ГБУ',
    'ГКУ',
    'ГУП',
    'Д/С',
    'ДСУ',
    'ДОУч',
    'ЖСК',
    'КБ',
    'КФХ',
    'МУУП',
    'МУУЧ',
    'МКУ',
    'МБУ',
    'НПО',
    'НПП',
    'НТЦ',
    'ОДО',
    'ПИФ',
    'ПРОФКОМ',
    'РедСМИ',
    'РСУ',
    'РЭУ',
    'СНТ',
    'СМУ',
    'ТСЖ',
    'ТД',
    'ФГУП',
    'ФКП',
    'ФБУ',
    'ФГУ',
    'ФГБУ',
    'ФКУ',
    'ЧОП',
    'ЧИФ',
    'Я/С',
])

TYPE = morph_pipeline([
    'б-ца',
    'з-д',
    'ин-т',
    'п-ка',
    
    'аптека',
    'магазин',
    'больница',
    'детский сад',
    'монастырь',
    'поликлиника',
    'церковь',
    'лицей',
    'ясли-сад',
    'нии',
    'академия',
    'обсерватория',
    'университет',
    'институт',
    'политех',
    'колледж',
    'техникум',
    'училище',
    'школа',
    'музей',
    'библиотека',
    
    'кооператив',
    'предприятие',
    'артель', 
    'ассоциация',
    'учреждение',
    'колхоз',
    'фирма',
    'фонд',
    'банк',
    'центр',
    'бюро',
    'товарищество',
    'отделение',
    'организация',
    'общество',
    'партия',
    'представительство',
    'приход',
    'община',
    'комитет',
    'совхоз',
    'филиал',
    'агентство',
    'компания',
    'издательство',
    'газета',
    'концерн',
    'завод',
    'корпорация',
    'группа компаний',
    'санаторий',
    'подразделение',
    
    'кафе',
    'ресторан',
    'закусочная',
    
    'авиакомпания',
    'госкомпания',
    'инвесткомпания',
    'медиакомпания',
    'оффшор-компания',
    'радиокомпания',
    'телекомпания',
    'телерадиокомпания',
    'траст-компания',
    'фактор-компания',
    'холдинг-компания',
    'энергокомпания',
    'компания-производитель',
    'компания-изготовитель',
    'компания-заказчик',
    'компания-исполнитель',
    'компания-посредник',
    'группа управляющих компаний',
    'агрофирма',
    'турфирма',
    'юрфирма',
    'фирма-производитель',
    'фирма-изготовитель',
    'фирма-заказчик',
    'фирма-исполнитель',
    'фирма-посредник',
    'авиапредприятие',
    'агропредприятие',
    'госпредприятие',
    'нацпредприятие',
    'промпредприятие',
    'энергопредприятие',
    'авиакорпорация',
    'госкорпорация',
    'профорганизация',
    'стартап',
    'нотариальная контора',
    'букмекерская контора',
    'авиазавод',
    'автозавод',
    'винзавод',
    'подстанция',
    'гидроэлектростанция',
    'общество',
    'акционерное общество',
    'открытое акционерное общество',
    'общество с ограниченной ответственностью',
    'закрытое акционерное общество',
    'публичное акционерное общество',
    'индивидуальный предприниматель',
    'некоммерческая организация',
    'адвокатская палата',
    'благотворительный фонд',
    'внебюджетный фонд',
    'государственное бюджетное учреждение',
    'государственное казенное учреждение',
    'государственное предприятие',
    'государственное унитарное предприятие',
    'государственное учреждение',
    'дачное товарищество',
    'дорожное строительное управление',
    'образовательное учреждение',
    'жилищно-строительный кооператив',
    'конструкторское бюро',
    'кредитный союз',
    'крестьянское фермерское хозяйство',
    'крестьянское хозяйство',
    'личное подсобное хозяйство',
    'малое предприятие',
    'межгосударственная финансово-промышленная группа',
    'муниципальное предприятие',
    'муниципальное казенное предприятие',
    'муниципальное унитарное предприятие',
    'муниципальное учреждение',
    'муниципальное казенное учреждение',
    'муниципальное бюджетное учреждение',
    'научно-производственная фирма',
    'научно-производственное объединение',
    'научно-производственное предприятие',
    'некоммерческое партнерство',
    'нотариальная контора',
    'нотариальная палата',
    'обособленное подразделение',
    'обособленное структурное подразделение',
    'объединение крестьянских фермерских хозяйств',
    'общероссийский профсоюз',
    'общественная организация',
    'общественное движение',
    'общественное объединение',
    'общественное учреждение',
    'общественный фонд',
    'общество с дополнительной ответственностью',
    'объединение предприятий',
    'объединение фермерских хозяйств',
    'орган общественной самодеятельности',
    'паевой инвестиционный фонд',
    'потребительский союз',
    'потребительское общество',
    'производственное объединение',
    'редакция средств массовой информации',
    'религиозная организация',
    'религиозное общество',
    'ремонтно-строительное управление',
    'ремонтно-эксплуатационное управление',
    'садоводческое товарищество', 
    'садоводческое некоммерческое товарищество', 
    'союз крестьянских фермерских хозяйств',
    'союз потребительских обществ',
    'строительно-монтажное управление',
    'структурное подразделение',
    'территориальная организация профсоюза',
    'территориальное общественное самоуправление',
    'товарищество на вере',
    'товарищество собственников жилья',
    'торговый дом',
    'управление производственно-технической комплектации',
    'управление делами',
    'федеральное государственное унитарное предприятие',
    'федеральное казенное предприятие',
    'федеральное бюджетное учреждение',
    'федеральное государственное учреждение',
    'федеральное государственное бюджетное учреждение',
    'федеральное казенное учреждение',
    'финансово-промышленная группа',
    'хозяйственное управление',
    'частное охранное предприятие',
    'чековый инвестиционный фонд',
])

In [7]:
Organisation = fact(
    'Organisation',
    ['descr', 'type', 'gent', 'name']
)

gnc = gnc_relation()
ADJF_PREFIX = rule(
    or_(
        and_(
            is_single(),
            gram('ADJF'),
            not_(gram('Apro')),
        ).match(gnc),  # международное
        rule(  # историко-просветительское
            and_(
                not_(type('LATIN')),
                not_(gram('NPRO')),
            ),
            eq('-'),
            and_(
                is_single(),
                gram('ADJF'),
                not_(gram('Apro')),
            ).match(gnc),
        ),
    ),
    or_(caseless('и'), eq(',')).optional(),
).repeatable(max=6).interpretation(Organisation.descr.inflected({'nomn'}))

ADJF_PREFIX_CAP = rule(
    or_(
        and_(
            is_capitalized(),
            is_single(),
            gram('ADJF'),
            not_(gram('Apro')),
        ).match(gnc),  # международное
        rule(  # историко-просветительское
            and_(
                not_(type('LATIN')),
                is_capitalized(),
                not_(gram('NPRO')),
            ),
            eq('-'),
            and_(
                is_single(),
                gram('ADJF'),
                not_(gram('Apro')),
            ).match(gnc),
        ),
    ),
    rule(
        or_(caseless('и'), eq(',')).optional(),
        or_(
            and_(
                is_single(),
                gram('ADJF'),
                not_(gram('Apro')),
            ).match(gnc),  # международное
            rule(  # историко-просветительское
                and_(
                    not_(type('LATIN')),
                    is_capitalized(),
                    not_(gram('NPRO')),
                ),
                eq('-'),
                and_(
                    is_single(),
                    gram('ADJF'),
                    not_(gram('Apro')),
                ).match(gnc),
            ),
        ),
    ).optional().repeatable(max=5)
).interpretation(Organisation.descr.inflected({'nomn'}).custom(lambda x: x.capitalize()))

ORGN_TYPE = rule(
    or_(
        ABBR_TYPE.interpretation(Organisation.type),
        TYPE.match(gnc).interpretation(Organisation.type.normalized()),
    )
)

case = case_relation()

GENT_GROUP = rule( # родительный падеж
    and_(
        gram('gent'),
        not_(gram('Abbr')),
        not_(gram('PREP')),
    ).match(case),
).repeatable(max=12).optional()

QUOTED_ORGN_NAME = or_(
            rule(
                in_(LEFT_QUOTES),
                not_(in_(RIGHT_QUOTES)).repeatable(max=20),
                in_(RIGHT_QUOTES),
            ),
            rule(
                in_(GENERAL_QUOTES),
                not_(in_(GENERAL_QUOTES)).repeatable(max=20),
                in_(GENERAL_QUOTES),
            ),
        )

CAPITALIZED_ORGN_NAME = rule(
    and_(
        is_capitalized(),
        not_(gram('PREP')),
        not_(gram('CONJ')),
    ).repeatable(max=6)
)

ORGN_NAME = rule(
    or_(
        QUOTED_ORGN_NAME,
        CAPITALIZED_ORGN_NAME,
    )
)

NAMED = rule(
    or_(
        rule(normalized('имя')),
        rule(caseless('им'), eq('.').optional()),
    ).optional(),
    or_(
        NAME,
        PERSON,
    ),
)

ORGN_AREA = rule(
    or_(
        RESPUBLIKA,
        KRAI,
        OBLAST,
        AUTO_OKRUG,
        RAION,
        GOROD,
        SELO,
        POSELOK,
        DEREVNYA,
    )
)

NUMERED = rule(
    eq('№'),
    type('INT'),
)

ORGN_ID = rule(
    or_(
        rule(
            NUMERED,
            or_(
                NAMED,
                ORGN_AREA,
                ORGN_NAME,
                GENT_GROUP,
            ).optional(),
        ),
        rule(
            NAMED,
            ORGN_NAME.optional(),
        ),
        rule(
            ORGN_AREA,
            ORGN_NAME.optional(),
        ),
        ORGN_NAME,
    ),
).interpretation(Organisation.name)


ORGANISATION = or_(
    rule(
        ADJF_PREFIX.optional(),
        ORGN_TYPE,
        GENT_GROUP.interpretation(Organisation.gent.custom(lambda x : x.lower())),
        ORGN_ID,
    ),
    rule(
        ADJF_PREFIX_CAP,
        ORGN_TYPE,
        GENT_GROUP.interpretation(Organisation.gent.custom(lambda x : x.lower())),
    ),
    
).interpretation(Organisation)

TOKENIZER = MorphTokenizer().remove_types('EOL')

orgn_parcer = Parser(ORGANISATION, tokenizer=TOKENIZER)
name_parcer = Parser(ABBR_TYPE, tokenizer=TOKENIZER)

In [8]:
soup = BeautifulSoup(res.content, "html.parser")
for script in soup(["script", "style"]):
    script.decompose()
    
concat_symbols_pattern = re.compile(r'[\u2010\u2011\u2012\u2013\u2014\u2015]')
text = soup.getText(separator='\n', strip=True)
text = concat_symbols_pattern.sub('-', text)

lines = [line for line in text.splitlines()]
# pprint(lines)
tmp_lines = list()
for line in lines:
    line = re.sub(r'\s+', ' ', line)
    if len(tmp_lines) == 0:
        tmp_lines.append(line)
    elif re.search(r'^([A-ZА-ЯЁ]|\d\.)', line) and not re.search(r'[:;, \xad\-«„ʼ"\'”]$', tmp_lines[-1]):
        tmp_lines.append(line)
    else:
        if re.search(r'[\xad«„ʼ]$', tmp_lines[-1]) or re.search(r'^[^\w\d«„ʼ"\'”]', line):
            tmp_lines[-1] = tmp_lines[-1] + line
        else:
            tmp_lines[-1] = tmp_lines[-1].strip() + ' ' + line

lines.clear()
for line in tmp_lines:
    line = line.replace('\xad', '')
    if not re.search(r'[:;?!.]$', line):
        lines.append(line + '.')
    else:
        lines.append(line)
            
text = '\n'.join(lines)

NameError: name 'res' is not defined

In [ ]:
print(text)

In [ ]:
text = "607130 Нижегородская обл Ардатовский р-н Ардатов рп Ленина ул 14"

In [ ]:
matches = area_parser.findall(text)
for m in matches:
    pprint(m.fact)

In [81]:
matches = extr(text)
for m in matches:
    pprint(m.fact)
#     print([_ for _ in m.tokens])

Контакты - Замена электросчетчика LVE Service.
Меню.
Электромонтажные работы.
Цены для юр.лиц.
О Компании.
Инфо.
Статьи.
Акции.
Тарифы на электроэнергию.
Контакты.
Контакты.
Контакты.
Контактный центр:+7 (985) 600-05-50.
Электронная почта: servicelve@gmail.com.
Фактический адрес: 109456 г. Москва, пр. Вешняковский 4й, д.8, стр.2.
Реквизиты.
Наименование компании: ООО «ЛВЕ Сервис».
Юридический адрес: 109456 г. Москва, пр. Вешняковский 4й, д.8, стр.2.
Телефон:+7 (985) 600-05-50.
ИНН/КПП: 9721035503/772101001.
Расчетный счет: 40702810570010012566.
Наименование банка: Московский филиал АО КБ «Модульбанк».
БИК: 044525092.
Кор.счет: 30101810645250000092.
ОГРН: 5167746511114.
Свежие записи.
Какой электросчетчик лучше купить?
Лучшие счетчики электрической энергии. Рейтинг ТОП-5.
Как заменить электросчетчик самостоятельно.
Отправка сведений о замене прибора в энергосбытовую организацию.
Порядок замены электросчетчика.
Свежие комментарии.
Архивы.
Август 2017.
Февраль 2017.
Январь 2017.
Декабрь 2

In [30]:
text = "607130 Нижегородская обл Ардатовский р-н Ардатов рп Ленина ул 14"

In [31]:
matches = area_parser.findall(text)
for m in matches:
    pprint(m.fact)

Area(name='Нижегородская обл')


In [11]:
matches = extr(text)
for m in matches:
    pprint(m.fact)
#     print([_ for _ in m.tokens])

Address(parts=[Street(name='Ленина', type='улица'), Building(number='14', type=None)])


In [ ]:
# ents = extr.entities_extraction(text)

In [ ]:
print(ents.cities)
print(ents.companies)